# 230713_analysis_hooking
각 submodule의 input output을 조사

In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer
from datasets import load_dataset
from tqdm import tqdm
import json
import torch
import argparse
import datasets
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pylab as plt
from os import listdir
import json
import math
import seaborn as sns 

/rds/general/user/jj1122/home/anaconda3/envs/m2d2_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Config

In [2]:
n_layers = 12
list_modules = ['attn', 'mlp']
trace_module_id = "transformer.h.{l}.{m}"

list_trace_module_ids = []

for l in range(n_layers):
    for m in list_modules:
        list_trace_module_ids.append(trace_module_id.format(l=l, m=m))

In [2]:
list_trace_module_ids = [
    'transformer.h.0.attn.c_attn', 'transformer.h.0.attn.c_proj',
    'transformer.h.0.mlp.c_fc', 'transformer.h.0.mlp.c_proj',
    'transformer.h.1.attn.c_attn', 'transformer.h.1.attn.c_proj',
    'transformer.h.1.mlp.c_fc', 'transformer.h.1.mlp.c_proj',
    'transformer.h.2.attn.c_attn', 'transformer.h.2.attn.c_proj',
    'transformer.h.2.mlp.c_fc', 'transformer.h.2.mlp.c_proj',
    'transformer.h.3.attn.c_attn', 'transformer.h.3.attn.c_proj',
    'transformer.h.3.mlp.c_fc', 'transformer.h.3.mlp.c_proj',
    'transformer.h.4.attn.c_attn', 'transformer.h.4.attn.c_proj',
    'transformer.h.4.mlp.c_fc', 'transformer.h.4.mlp.c_proj',
    'transformer.h.5.attn.c_attn', 'transformer.h.5.attn.c_proj',
    'transformer.h.5.mlp.c_fc', 'transformer.h.5.mlp.c_proj',
    'transformer.h.6.attn.c_attn', 'transformer.h.6.attn.c_proj',
    'transformer.h.6.mlp.c_fc', 'transformer.h.6.mlp.c_proj',
    'transformer.h.7.attn.c_attn', 'transformer.h.7.attn.c_proj',
    'transformer.h.7.mlp.c_fc', 'transformer.h.7.mlp.c_proj',
    'transformer.h.8.attn.c_attn', 'transformer.h.8.attn.c_proj',
    'transformer.h.8.mlp.c_fc', 'transformer.h.8.mlp.c_proj',
    'transformer.h.9.attn.c_attn', 'transformer.h.9.attn.c_proj',
    'transformer.h.9.mlp.c_fc', 'transformer.h.9.mlp.c_proj',
    'transformer.h.10.attn.c_attn', 'transformer.h.10.attn.c_proj',
    'transformer.h.10.mlp.c_fc', 'transformer.h.10.mlp.c_proj',
    'transformer.h.11.attn.c_attn', 'transformer.h.11.attn.c_proj',
    'transformer.h.11.mlp.c_fc', 'transformer.h.11.mlp.c_proj'
]

In [59]:
for tup in gpt2.named_modules():
    print(tup[0])


transformer
transformer.wte
transformer.wpe
transformer.drop
transformer.h
transformer.h.0
transformer.h.0.ln_1
transformer.h.0.attn
transformer.h.0.attn.c_attn
transformer.h.0.attn.c_proj
transformer.h.0.attn.attn_dropout
transformer.h.0.attn.resid_dropout
transformer.h.0.ln_2
transformer.h.0.mlp
transformer.h.0.mlp.c_fc
transformer.h.0.mlp.c_proj
transformer.h.0.mlp.act
transformer.h.0.mlp.dropout
transformer.h.1
transformer.h.1.ln_1
transformer.h.1.attn
transformer.h.1.attn.c_attn
transformer.h.1.attn.c_proj
transformer.h.1.attn.attn_dropout
transformer.h.1.attn.resid_dropout
transformer.h.1.ln_2
transformer.h.1.mlp
transformer.h.1.mlp.c_fc
transformer.h.1.mlp.c_proj
transformer.h.1.mlp.act
transformer.h.1.mlp.dropout
transformer.h.2
transformer.h.2.ln_1
transformer.h.2.attn
transformer.h.2.attn.c_attn
transformer.h.2.attn.c_proj
transformer.h.2.attn.attn_dropout
transformer.h.2.attn.resid_dropout
transformer.h.2.ln_2
transformer.h.2.mlp
transformer.h.2.mlp.c_fc
transformer.h.2.mlp

In [3]:
gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokeniser = GPT2Tokenizer.from_pretrained("gpt2")

In [4]:
def tokenize_function(examples):
    output = gpt2_tokeniser(examples)
    return output

In [5]:
temp = {}

def func_v1(m_id):
    def hook_v1(module, _input, _output):
        global seq
        dict_sequence[m_id].append(seq)
        seq += 1
        
        module_ = list(module) if isinstance(module, tuple) else module
        input_ = list(_input) if isinstance(_input, tuple) else _input
        output_ = list(_output) if isinstance(_output, tuple) else _output
#         print(type(module_), module_)
        temp[m_id] = [module_, input_, output_]
        
    return hook_v1

gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")

for m_id in list_trace_module_ids:
    gpt2.get_submodule(m_id).register_forward_hook(func_v1(m_id))    


In [88]:
input_text = ["hi hello you", 'good bye you']
data = tokenize_function(input_text)
inputs = torch.tensor(data['input_ids'])
inputs

tensor([[ 5303, 23748,   345],
        [11274, 33847,   345]])

In [89]:
dict_sequence = {x: [] for x in list_trace_module_ids}
seq = 0

with torch.no_grad():
    clean_outputs = gpt2(inputs, labels=inputs.clone())
    clean_loss = np.exp(clean_outputs.loss.item())


## Order

In [90]:
dict_sequence

{'transformer.h.0.attn.c_attn': [0],
 'transformer.h.0.attn.c_proj': [1],
 'transformer.h.0.mlp.c_fc': [2],
 'transformer.h.0.mlp.c_proj': [3],
 'transformer.h.1.attn.c_attn': [4],
 'transformer.h.1.attn.c_proj': [5],
 'transformer.h.1.mlp.c_fc': [6],
 'transformer.h.1.mlp.c_proj': [7],
 'transformer.h.2.attn.c_attn': [8],
 'transformer.h.2.attn.c_proj': [9],
 'transformer.h.2.mlp.c_fc': [10],
 'transformer.h.2.mlp.c_proj': [11],
 'transformer.h.3.attn.c_attn': [12],
 'transformer.h.3.attn.c_proj': [13],
 'transformer.h.3.mlp.c_fc': [14],
 'transformer.h.3.mlp.c_proj': [15],
 'transformer.h.4.attn.c_attn': [16],
 'transformer.h.4.attn.c_proj': [17],
 'transformer.h.4.mlp.c_fc': [18],
 'transformer.h.4.mlp.c_proj': [19],
 'transformer.h.5.attn.c_attn': [20],
 'transformer.h.5.attn.c_proj': [21],
 'transformer.h.5.mlp.c_fc': [22],
 'transformer.h.5.mlp.c_proj': [23],
 'transformer.h.6.attn.c_attn': [24],
 'transformer.h.6.attn.c_proj': [25],
 'transformer.h.6.mlp.c_fc': [26],
 'transform

# ATTN

## attn_c_attn

In [91]:
attn_c_attn = "transformer.h.0.attn.c_attn"

### module

In [92]:
temp[attn_c_attn][0]

Conv1D()

In [93]:
temp[attn_c_attn][0].weight.shape, temp[attn_c_attn][0].bias.shape

(torch.Size([768, 2304]), torch.Size([2304]))

### input

In [94]:
temp[attn_c_attn][1]

[tensor([[[-0.1000, -0.0874, -0.0269,  ..., -0.0154,  0.0138,  0.1415],
          [ 0.1052, -0.1196, -0.0540,  ..., -0.1193,  0.0812,  0.0517],
          [-0.0343, -0.0042,  0.0058,  ..., -0.1106, -0.0695, -0.0866]],
 
         [[ 0.0065, -0.0714, -0.0597,  ..., -0.0201, -0.0945, -0.0281],
          [ 0.0837, -0.2269, -0.0595,  ..., -0.3107, -0.1329,  0.0581],
          [-0.0343, -0.0042,  0.0058,  ..., -0.1106, -0.0695, -0.0866]]])]

In [95]:
temp[attn_c_attn][1][0].shape

torch.Size([2, 3, 768])

### output (activation, (k, v))

In [96]:
temp[attn_c_attn][2]

tensor([[[ 0.6655, -0.2217,  0.1277,  ...,  0.0662, -0.0129,  0.0362],
         [ 0.4013,  0.8140,  0.4187,  ..., -0.0534, -0.0017,  0.3631],
         [-0.0061, -0.6571, -0.9560,  ...,  0.0841,  0.1047,  0.2007]],

        [[-0.0903, -0.4669, -0.4292,  ..., -0.0223, -0.0383,  0.0397],
         [-0.0209,  0.3601, -0.9751,  ...,  0.0634, -0.1967,  0.4776],
         [-0.0061, -0.6571, -0.9560,  ...,  0.0841,  0.1047,  0.2007]]])

In [97]:
temp[attn_c_attn][2].shape

torch.Size([2, 3, 2304])

## attn_c_proj

In [98]:
attn_c_proj = "transformer.h.0.attn.c_proj"

### module

In [99]:
temp[attn_c_proj][0].weight.shape, temp[attn_c_proj][0].bias.shape

(torch.Size([768, 768]), torch.Size([768]))

### input

In [100]:
temp[attn_c_proj][1]

[tensor([[[ 0.1432, -0.0883, -0.1866,  ...,  0.0662, -0.0129,  0.0362],
          [ 0.1754, -0.0284, -0.1838,  ..., -0.0015, -0.0066,  0.2211],
          [ 0.1662,  0.0007, -0.1727,  ...,  0.0411,  0.0261,  0.1704]],
 
         [[ 0.1572, -0.0891,  0.0698,  ..., -0.0223, -0.0383,  0.0397],
          [ 0.1471, -0.0393,  0.1088,  ...,  0.0210, -0.1183,  0.2609],
          [ 0.1256,  0.0327,  0.1329,  ...,  0.0261, -0.0372,  0.1851]]])]

In [101]:
temp[attn_c_proj][1][0].shape

torch.Size([2, 3, 768])

### output (activation, (k, v))

In [102]:
temp[attn_c_proj][2]

tensor([[[ 1.1389, -1.3163, -0.0550,  ...,  0.0416,  0.0390, -0.0281],
         [ 0.7931, -0.4850, -0.4926,  ...,  0.0448,  0.0371, -0.0997],
         [-0.1901,  0.0686, -1.0865,  ...,  0.0132,  0.0369,  0.0170]],

        [[ 0.6993, -1.0576,  0.3251,  ...,  0.0449, -0.0214,  0.0940],
         [ 2.2979, -1.0934, -0.0447,  ...,  0.0073, -0.0284,  0.0597],
         [-0.1887, -0.4862, -0.6547,  ..., -0.0183, -0.0460, -0.0433]]])

In [103]:
temp[attn_c_proj][2].shape

torch.Size([2, 3, 768])

# MLP

## mlp_c_fc

### module

In [105]:
mlp_c_fc = "transformer.h.0.mlp.c_fc"

In [106]:
temp[mlp_c_fc][0]

Conv1D()

In [107]:
temp[mlp_c_fc][0].weight.shape, temp[mlp_c_fc][0].bias.shape

(torch.Size([768, 3072]), torch.Size([3072]))

### input

In [108]:
temp[mlp_c_fc][1]

[tensor([[[ 1.4949e-01, -2.3579e-01,  1.3381e-02,  ...,  3.1491e-02,
            1.2534e-01,  2.7030e-01],
          [ 1.2421e-01, -6.7010e-02, -6.6431e-02,  ..., -1.0497e-01,
            1.3507e-01, -2.5451e-02],
          [ 1.5652e-02,  3.4081e-02, -1.7358e-01,  ..., -1.4329e-01,
           -4.6414e-02, -7.6008e-02]],
 
         [[ 1.2513e-01, -1.9504e-01,  6.6614e-02,  ...,  2.4091e-02,
           -1.9277e-01,  7.2787e-02],
          [ 2.5648e-01, -1.7055e-01, -7.6198e-05,  ..., -3.4101e-01,
           -1.5102e-01,  1.1415e-01],
          [ 1.5357e-02, -6.5804e-02, -1.0100e-01,  ..., -1.7909e-01,
           -1.3713e-01, -1.3167e-01]]])]

In [109]:
temp[mlp_c_fc][1][0].shape

torch.Size([2, 3, 768])

### output

In [110]:
temp[mlp_c_fc][2]

tensor([[[ 0.4491, -1.5715, -1.4653,  ..., -2.0566, -2.0355,  0.6436],
         [ 0.6902,  0.1128,  0.2437,  ..., -1.0926, -1.4637, -0.1455],
         [ 0.5122, -1.0178, -0.7147,  ..., -1.7263, -0.9549, -0.0196]],

        [[-0.0673, -1.6470, -1.0625,  ..., -1.9722, -1.9229,  0.3057],
         [ 0.5578,  0.2786, -0.1493,  ..., -1.3881, -0.5546,  0.1224],
         [ 0.4659, -1.1941, -0.6374,  ..., -1.5148, -0.9111, -0.2381]]])

In [111]:
temp[mlp_c_fc][2].shape

torch.Size([2, 3, 3072])

## mlp_c_proj

### module

In [112]:
mlp_c_proj = "transformer.h.0.mlp.c_proj"

In [113]:
temp[mlp_c_proj][0]

Conv1D()

In [114]:
temp[mlp_c_proj][0].weight.shape, temp[mlp_c_proj][0].bias.shape

(torch.Size([3072, 768]), torch.Size([768]))

### input

In [115]:
temp[mlp_c_proj][1]

[tensor([[[ 0.3024, -0.0914, -0.1049,  ..., -0.0407, -0.0424,  0.4763],
          [ 0.5210,  0.0614,  0.1453,  ..., -0.1502, -0.1051, -0.0643],
          [ 0.3564, -0.1573, -0.1697,  ..., -0.0729, -0.1623, -0.0096]],
 
         [[-0.0318, -0.0821, -0.1532,  ..., -0.0478, -0.0524,  0.1896],
          [ 0.3968,  0.1699, -0.0658,  ..., -0.1148, -0.1606,  0.0672],
          [ 0.3165, -0.1390, -0.1670,  ..., -0.0985, -0.1651, -0.0966]]])]

In [116]:
temp[mlp_c_proj][1][0].shape

torch.Size([2, 3, 3072])

### output

In [117]:
temp[mlp_c_proj][2]

tensor([[[-0.1473, -0.8551,  0.2517,  ..., -1.0792,  0.4386, -0.8061],
         [-0.9995, -0.4823, -1.1081,  ..., -0.7381, -0.0529, -1.1930],
         [-0.9196,  0.3858, -0.1840,  ..., -0.8790, -0.0025,  0.8709]],

        [[-0.0233, -0.2503, -0.6902,  ..., -0.6061,  0.2298, -0.5997],
         [ 1.0200, -1.0674, -2.2112,  ..., -0.7643, -0.2864,  0.3234],
         [-1.2876,  0.6676, -0.1443,  ..., -1.0138,  0.1746,  0.9701]]])

In [118]:
temp[mlp_c_proj][2].shape

torch.Size([2, 3, 768])